In [1]:
pip install --upgrade yfinance

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
bs = yf.Ticker("AZN").quarterly_balance_sheet

#no seasonality is assumed for biotech
terminal_growth = 0.01
federal_rate = 1.0428
market_growth = 1.113
DKK_TO_USD = 0.14
JPY_TO_USD = 0.0066

In [2]:
ticker_list = ["AZN", "LLY", "NVO", "TAK", "BMY"] #, "PFE", "GSK", "SNY", "ROG.SW", "TAK", "NVO", "NVS", "GILD"

beta_dict = {}
for ticker in ticker_list:
    try:
        ticker_info = yf.Ticker(ticker).info
        beta = ticker_info.get("beta")
        if beta is not None:
            beta_dict[ticker] = beta
        else:
            print(f"Beta information not available for {ticker}")
    except Exception as e:
        print(f"An error occurred while fetching data for {ticker}: {e}")

beta_df = pd.DataFrame(list(beta_dict.items()), columns=['Ticker', 'Beta'])
beta_df.set_index('Ticker', inplace=True)
beta_df


,Beta
Ticker,
AZN,0.201
LLY,0.412
NVO,0.192
TAK,0.456
BMY,0.451


In [3]:
# prompt: make a dataframe with Close prices for all companies in ticker_list for the 5 most recent dates in columns of bs dataframe - use the 5 recent most dates that appear as columns in bs - if the date is not in the prices data, pick the closest available date to it - only put the prices in the dataframe

# Get the 5 most recent dates from the columns of the 'bs' DataFrame.
dates = bs.columns[:5]

# Create an empty dictionary to store the prices for each company.
prices_dict = {}

# Loop through each ticker in the ticker_list.
for ticker in ticker_list:
    # Download the historical data for the current ticker.
    data = yf.download(ticker, period="max")

    # Create an empty list to store the prices for the current ticker.
    prices = []

    # Loop through each date in the 'dates' list.
    for date in dates:
        # Convert the date to a datetime object.
        date_dt = pd.to_datetime(date)

        # Find the closest available date in the historical data.
        closest_date = min(data.index, key=lambda x: abs(x - date_dt))

        # Get the closing price for the closest date.
        price = data.loc[closest_date, 'Close'].values[0]

        # Append the price to the 'prices' list.
        prices.append(price)

    # Store the prices for the current ticker in the 'prices_dict'.
    prices_dict[ticker] = prices

# Create a DataFrame from the 'prices_dict'.
prices_df = pd.DataFrame(prices_dict, index=dates)

# Display the DataFrame.
prices_df


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,AZN,LLY,NVO,TAK,BMY
2024-12-31,64.597305,770.671753,86.019997,13.24,55.942513
2024-09-30,76.812828,882.953430,119.070000,14.22,50.608620
2024-06-30,76.372993,910.015991,144.877731,13.00,39.791859
2024-03-31,65.897919,755.640564,127.074371,13.67,50.467918
2023-12-31,64.997200,578.143738,102.325958,14.27,48.307255


In [4]:
balance_sheet_map = {
    'Ordinary Shares Number':'Ordinary Shares Number',
    'Cash Cash Equivalents And Short Term Investments':'Cash & Short Term Investments',
    'Accounts Receivable':'Accounts Receivable',
    'Inventory':'Inventory',
    'Current Assets':'Current Assets',
    'Total Assets':'Assets',
    'Total Liabilities Net Minority Interest':'Total Liabilities',
    'Accounts Payable':'Accounts Payable',
    'Long Term Debt And Capital Lease Obligation':'Long Term Debt And Capital Lease Obligation',
    'Current Debt And Capital Lease Obligation':'Current Debt And Capital Lease Obligation',
    'Current Liabilities':'Total Current Liabilities',
    'Stockholders Equity':'Total Equity'
}

income_statement_map = {
    'Total Revenue':'Revenue',
    'Cost Of Revenue':'Cost of Revenue',
    'Gross Profit':'Gross Profit',
    'EBITDA':'EBITDA',
    'EBIT':'EBIT',
    'Interest Expense':'Interest Expense',
    'Income Tax Expense':'Income Tax Expense',
    'Net Income':'Net Income',
    'Net Income Continuous Operations':'Net Income Continuous Operations',
    'Basic EPS':'Basic EPS',
    'Tax Rate For Calcs':'Tax Rate For Calcs',
    #'Basic Average Shares':'Basic Average Shares'
}

cash_flow_map = {
    'Operating Cash Flow':'Operating Cash Flow',
    'Capital Expenditures':'Capital Expenditures',
    'Free Cash Flow':'Free Cash Flow',
    'Cash Dividends Paid': 'Cash Dividends Paid'
}

In [133]:
fin_all = pd.DataFrame()

for ticker in ticker_list:
  print(ticker)

  multiplier = 1
  if ticker == 'TAK':
    multiplier = JPY_TO_USD
    terminal_growth = 0.005
  if ticker == 'NVO':
    multiplier = DKK_TO_USD

  bs = yf.Ticker(ticker).quarterly_balance_sheet
  bs = bs[bs.index.isin(balance_sheet_map.keys())]
  bs = bs.sort_index(axis=1, ascending=False)
  #multiple all number fields with multiplier in bs
  bs = bs.apply(lambda x: x * multiplier if x.name in list(balance_sheet_map.keys())[1:-1] else x)

  income = yf.Ticker(ticker).quarterly_income_stmt
  income = income[income.index.isin(income_statement_map.keys())]
  income = income.sort_index(axis=1, ascending=False)
  #multiple all number keys with multiplier in income
  income = income.apply(lambda x: x * multiplier if x.name in list(income_statement_map.keys())[:-1] else x)

  cf = yf.Ticker(ticker).quarterly_cash_flow
  cf = cf[cf.index.isin(cash_flow_map.keys())]
  cf = cf.sort_index(axis=1, ascending=False)
  cf = cf.apply(lambda x: x * multiplier if x.name in list(cash_flow_map.keys()) else x)

  fin = pd.concat([bs, cf, income], axis=0)
  fin = fin.sort_index(axis=1, ascending=False)
  fin.loc['Close price', :] = prices_df.loc[:, ticker]

  #fin.dropna(axis=1,inplace=True)
  fin = fin.drop(columns=[col for col in fin if col not in prices_df.index])

  fin = fin.T
  fin['Market cap'] = fin['Close price'] * fin['Ordinary Shares Number']
  fin['Debt to equity'] = fin['Total Liabilities Net Minority Interest'] / fin['Stockholders Equity']
  fin['Interest coverage'] = fin['EBITDA'] / fin['Interest Expense']
  fin['Current ratio'] = fin['Current Assets'] / fin['Current Liabilities']
  fin['Cash ratio'] = fin['Cash Cash Equivalents And Short Term Investments'] / fin['Current Liabilities']
  fin['Sales to assets'] = fin['Total Revenue'] / fin['Total Assets']
  fin['Days in inventory'] = 365 * fin['Inventory'] / fin['Cost Of Revenue']
  fin['Account receiveable days'] = 365 * fin['Accounts Receivable'] / fin['Total Revenue']
  fin['Account payable days'] = 365 * fin['Accounts Payable'] / fin['Cost Of Revenue']
  fin['Gross profit margin'] = fin['Gross Profit'] / fin['Total Revenue']
  fin['EBITDA margin'] = fin['EBITDA'] / fin['Total Revenue']
  fin['EBIT margin'] = fin['EBIT'] / fin['Total Revenue']
  fin['Profit margin from continuing operations'] = fin['Net Income Continuous Operations'] / fin['Total Revenue']
  fin['Net profit margin'] = fin['Net Income'] / fin['Total Revenue']
  fin['Return on Assets'] = fin['Net Income'] / fin['Total Revenue']
  fin['Cash Dividends Paid'] = np.abs(fin['Cash Dividends Paid'])
  fin['Dividend per share'] = fin['Cash Dividends Paid'] / fin['Ordinary Shares Number']
  fin['Payout ratio'] = fin['Cash Dividends Paid'] / fin['Net Income']
  fin['Return on Equity'] = fin['Net Income'] / fin['Stockholders Equity']
  fin['Basic EPS'] = np.where(fin['Basic EPS'].isna(), fin['Net Income'] / fin['Ordinary Shares Number'], fin['Basic EPS'])
  fin['P/E'] = fin['Close price'] / fin['Basic EPS']
  fin['Dividend yield'] = fin['Dividend per share'] / fin['Close price']
  fin['Beta'] = beta_df.loc[ticker].values[0]
  revenue_gr = (fin['Total Revenue'].diff(-1).shift(1) / fin['Total Revenue']).mean()
  if (fin['Cash Dividends Paid']>0).all():
    dividend_gr = (fin['Dividend per share'].diff(-1).shift(1) / fin['Dividend per share']).mean()
  else:
    dividend_gr = 0
  fcf_gr = (fin['Free Cash Flow'].diff(-1).shift(1) / fin['Free Cash Flow']).mean()
  fin['RevGR'] = fin['Total Revenue'].diff(-1).shift(1) / fin['Total Revenue']
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
  if (fin['Cash Dividends Paid']>0).all():
    fin['DivGR'] = fin['Dividend per share'].diff(-1).shift(1) / fin['Dividend per share']
  else:
    fin['DivGR'] = 0
  fin['DivGR'] = fin['DivGR'].fillna(dividend_gr)
  fin['FCFGR'] = fin['Free Cash Flow'].diff(-1).shift(1) / fin['Free Cash Flow']
  fin['FCFGR'] = fin['FCFGR'].fillna(fcf_gr)
  #convert to yearly
  fin['RevGR'] = fin['RevGR'] * 4
  fin['DivGR'] = fin['DivGR'] * 4
  fin['FCFGR'] = fin['FCFGR'] * 4
  fin['rE(Beta)'] = federal_rate + fin['Beta']*(market_growth-federal_rate)-1 #CAPM
  fin['rE(Dividend)'] = fin['Dividend per share']*(1+fin['DivGR'])/fin['Close price'] + fin['DivGR']
  fin['rE'] = np.where((fin['rE(Dividend)']>0.03) & (fin['rE(Dividend)']<0.15), fin['rE(Dividend)'], fin['rE(Beta)'])
  fin['Total debt'] = fin['Current Debt And Capital Lease Obligation'] + fin['Long Term Debt And Capital Lease Obligation']
  fin['rD'] = fin['Interest Expense'] / fin['Total debt']
  #getting rid of negatives
  fin['rD'] = np.where(fin['rD'] > 0, fin['rD'], fin['rD'].shift(-1))
  fin['rD'] = np.where(fin['rD'].isna(), fin['rD'].shift(1), fin['rD'])
  fin['Free Cash Flow'] = np.where(fin['Free Cash Flow'] > 0, fin['Free Cash Flow'], fin['Free Cash Flow'].shift(-1))
  fin['Free Cash Flow'] = np.where(fin['Free Cash Flow'].isnull(), fin['Free Cash Flow'].shift(1), fin['Free Cash Flow'])
  fin['wE'] = fin['Market cap'] / (fin['Market cap'] + fin['Total debt'])
  fin['wD'] = 1 - fin['wE']
  fin['WACC'] = fin['wE'] * fin['rE'] + fin['wD'] * fin['rD'] * (1-fin['Tax Rate For Calcs'])
  fin['Growth'] = np.where( (fin['RevGR'] > terminal_growth) & (fin['FCFGR'] > terminal_growth), np.where(fin['RevGR'] > fin['FCFGR'], fin['FCFGR'], fin['RevGR']), np.where(fin['RevGR'] > terminal_growth, fin['RevGR'], terminal_growth) )
  fin['RollingFCF'] = fin['Free Cash Flow'][::-1].rolling(2).mean()[::-1].fillna(fin['Free Cash Flow'])
  fin['FCF6'] = fin['RollingFCF'] * 4 * (1+fin['Growth'])**6 #*4 to convert to yearly
  fin['Discounted terminal value'] = (fin['FCF6'] / (fin['WACC']-terminal_growth)) / (1+fin['WACC'])**5
  fin['Growing annuity'] = fin['RollingFCF']*(1+fin['Growth'])/(fin['WACC']-fin['Growth']) * (1-((1+fin['Growth'])/(1+fin['WACC']))**5)
  fin['Total value now'] = fin['Discounted terminal value'] + fin['Growing annuity']
  fin['DCF fair share value'] = fin['Total value now'] / fin['Ordinary Shares Number']
  fin['Share px relative to fair value'] = fin['Close price'] / fin['DCF fair share value']

  fin['ticker'] = ticker
  fin = fin.reset_index()
  fin.rename(columns={'index': 'Date'}, inplace=True)

  fin_all = pd.concat([fin_all, fin], axis=0)
fin = fin_all.set_index('ticker')

AZN


<ipython-input-133-e4a204cf2d72>:68: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
<ipython-input-133-e4a204cf2d72>:73: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['DivGR'] = fin['DivGR'].fillna(dividend_gr)
<ipython-input-133-e4a204cf2d72>:75: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)

LLY


<ipython-input-133-e4a204cf2d72>:68: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
<ipython-input-133-e4a204cf2d72>:73: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['DivGR'] = fin['DivGR'].fillna(dividend_gr)
<ipython-input-133-e4a204cf2d72>:75: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)

NVO


<ipython-input-133-e4a204cf2d72>:68: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
<ipython-input-133-e4a204cf2d72>:75: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['FCFGR'] = fin['FCFGR'].fillna(fcf_gr)
<ipython-input-133-e4a204cf2d72>:94: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  f

TAK
BMY


<ipython-input-133-e4a204cf2d72>:68: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
<ipython-input-133-e4a204cf2d72>:73: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['DivGR'] = fin['DivGR'].fillna(dividend_gr)
<ipython-input-133-e4a204cf2d72>:75: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)

In [134]:
fin

,Date,Ordinary Shares Number,Stockholders Equity,Total Liabilities Net Minority Interest,Long Term Debt And Capital Lease Obligation,Current Liabilities,Current Debt And Capital Lease Obligation,Accounts Payable,Total Assets,Current Assets,...,wD,WACC,Growth,RollingFCF,FCF6,Discounted terminal value,Growing annuity,Total value now,DCF fair share value,Share px relative to fair value
ticker,,,,,,,,,,,,,,,,,,,,,
AZN,2024-12-31,1550546239.0,40786000000.0,63164000000.0,27619000000.0,27866000000.0,2495000000.0,22465000000.0,104035000000.0,25827000000.0,...,0.231157,0.04672,0.221115,1.988500e+09,2.637094e+10,503071499431.159363,16163490179.919909,519234989611.079285,334.872303,0.192901
AZN,2024-09-30,1550294658.0,40719000000.0,64117000000.0,29992000000.0,25926000000.0,1570000000.0,21684000000.0,104922000000.0,24155000000.0,...,0.209513,0.047353,0.391006,2.221000e+09,6.435511e+10,1205683953972.901855,28158202648.372829,1233842156621.274658,795.8759,0.096514
AZN,2024-06-30,1550268075.0,39512000000.0,64742000000.0,28174000000.0,28566000000.0,5359000000.0,20463000000.0,104340000000.0,25393000000.0,...,0.220711,0.046682,0.193848,1.649000e+09,1.909739e+10,364712130273.010681,12447474538.440491,377159604811.451172,243.2867,0.313922
AZN,2024-03-31,1550221288.0,37445000000.0,64792000000.0,28220000000.0,28901000000.0,6331000000.0,19699000000.0,102293000000.0,25594000000.0,...,0.252737,0.044883,0.081710,1.157500e+09,7.417300e+09,149319213844.576111,6428963788.896741,155748177633.472839,100.468352,0.655907
AZN,2023-12-31,1550162626.0,39143000000.0,61953000000.0,23222000000.0,30542000000.0,5400000000.0,3267000000.0,101119000000.0,25054000000.0,...,0.221227,0.047519,0.217898,1.434000e+09,1.871861e+10,349046009675.821167,11526063441.058678,360572073116.879822,232.602739,0.279434
LLY,2024-12-31,897538000.0,14192100000.0,64443300000.0,28527100000.0,28376600000.0,5117100000.0,3228600000.0,78714900000.0,32739700000.0,...,0.046383,0.068667,0.421599,3.950000e+08,1.304126e+10,146958626156.077087,5036561712.033978,151995187868.111053,169.3468,4.550849
LLY,2024-09-30,899850000.0,14240000000.0,61286200000.0,29045400000.0,24674100000.0,2074300000.0,2886500000.0,75606900000.0,31415100000.0,...,0.037691,0.069162,0.732121,6.340000e+07,6.848854e+09,76405599674.291367,1683002841.926961,78088602516.218323,86.779577,10.174668
LLY,2024-06-30,900416000.0,13562000000.0,58239300000.0,23730400000.0,27121200000.0,5161600000.0,2924800000.0,71874800000.0,30204300000.0,...,0.034059,0.069462,0.048236,7.330000e+07,3.889717e+08,4313056640.886327,345246374.079828,4658303014.966155,5.173501,175.899454
LLY,2024-03-31,900403000.0,12812200000.0,51046100000.0,24559900000.0,18598100000.0,1651500000.0,2473700000.0,63943500000.0,25188800000.0,...,0.037096,0.069287,1.156387,8.320000e+07,3.346134e+10,372351450539.833984,5339785679.264625,377691236219.098633,419.4691,1.801421


In [135]:
fin_all.to_csv('fin_all.csv')

In [138]:
#sanity check
fin_all.reset_index()[['ticker', 'Date']].value_counts().sort_values()

ticker  Date      
AZN     2023-12-31    1
        2024-09-30    1
        2024-12-31    1
BMY     2023-12-31    1
        2024-03-31    1
        2024-06-30    1
        2024-09-30    1
        2024-12-31    1
LLY     2023-12-31    1
        2024-03-31    1
AZN     2024-03-31    1
        2024-06-30    1
LLY     2024-06-30    1
NVO     2023-12-31    1
        2024-03-31    1
        2024-06-30    1
        2024-09-30    1
        2024-12-31    1
TAK     2023-12-31    1
        2024-03-31    1
        2024-06-30    1
        2024-09-30    1
LLY     2024-09-30    1
        2024-12-31    1
TAK     2024-12-31    1
Name: count, dtype: int64

In [128]:
fin_all = pd.DataFrame()

for ticker in ['BMY']:

  multiplier = 1
  if ticker == 'TAK':
    multiplier = JPY_TO_USD
    terminal_growth = 0.005
  if ticker == 'NVO':
    multiplier = DKK_TO_USD

  bs = yf.Ticker(ticker).quarterly_balance_sheet
  bs = bs[bs.index.isin(balance_sheet_map.keys())]
  bs = bs.sort_index(axis=1, ascending=False)
  #multiple all number fields with multiplier in bs
  bs = bs.apply(lambda x: x * multiplier if x.name in list(balance_sheet_map.keys())[1:-1] else x)

  income = yf.Ticker(ticker).quarterly_income_stmt
  income = income[income.index.isin(income_statement_map.keys())]
  income = income.sort_index(axis=1, ascending=False)
  #multiple all number keys with multiplier in income
  income = income.apply(lambda x: x * multiplier if x.name in list(income_statement_map.keys())[:-1] else x)

  cf = yf.Ticker(ticker).quarterly_cash_flow
  cf = cf[cf.index.isin(cash_flow_map.keys())]
  cf = cf.sort_index(axis=1, ascending=False)
  cf = cf.apply(lambda x: x * multiplier if x.name in list(cash_flow_map.keys()) else x)

  fin = pd.concat([bs, cf, income], axis=0)
  fin = fin.sort_index(axis=1, ascending=False)
  fin.loc['Close price', :] = prices_df.loc[:, ticker]

  #fin.dropna(axis=1,inplace=True)
  fin = fin.drop(columns=[col for col in fin if col not in prices_df.index])

  fin = fin.T
  fin['Market cap'] = fin['Close price'] * fin['Ordinary Shares Number']
  fin['Debt to equity'] = fin['Total Liabilities Net Minority Interest'] / fin['Stockholders Equity']
  fin['Interest coverage'] = fin['EBITDA'] / fin['Interest Expense']
  fin['Current ratio'] = fin['Current Assets'] / fin['Current Liabilities']
  fin['Cash ratio'] = fin['Cash Cash Equivalents And Short Term Investments'] / fin['Current Liabilities']
  fin['Sales to assets'] = fin['Total Revenue'] / fin['Total Assets']
  fin['Days in inventory'] = 365 * fin['Inventory'] / fin['Cost Of Revenue']
  fin['Account receiveable days'] = 365 * fin['Accounts Receivable'] / fin['Total Revenue']
  fin['Account payable days'] = 365 * fin['Accounts Payable'] / fin['Cost Of Revenue']
  fin['Gross profit margin'] = fin['Gross Profit'] / fin['Total Revenue']
  fin['EBITDA margin'] = fin['EBITDA'] / fin['Total Revenue']
  fin['EBIT margin'] = fin['EBIT'] / fin['Total Revenue']
  fin['Profit margin from continuing operations'] = fin['Net Income Continuous Operations'] / fin['Total Revenue']
  fin['Net profit margin'] = fin['Net Income'] / fin['Total Revenue']
  fin['Return on Assets'] = fin['Net Income'] / fin['Total Revenue']
  fin['Cash Dividends Paid'] = np.abs(fin['Cash Dividends Paid'])
  fin['Dividend per share'] = fin['Cash Dividends Paid'] / fin['Ordinary Shares Number']
  fin['Payout ratio'] = fin['Cash Dividends Paid'] / fin['Net Income']
  fin['Return on Equity'] = fin['Net Income'] / fin['Stockholders Equity']
  fin['Basic EPS'] = np.where(fin['Basic EPS'].isna(), fin['Net Income'] / fin['Ordinary Shares Number'], fin['Basic EPS'])
  fin['P/E'] = fin['Close price'] / fin['Basic EPS']
  fin['Dividend yield'] = fin['Dividend per share'] / fin['Close price']
  fin['Beta'] = beta_df.loc[ticker].values[0]
  revenue_gr = (fin['Total Revenue'].diff(-1).shift(1) / fin['Total Revenue']).mean()
  if (fin['Cash Dividends Paid']>0).all():
    dividend_gr = (fin['Dividend per share'].diff(-1).shift(1) / fin['Dividend per share']).mean()
  else:
    dividend_gr = 0
  fcf_gr = (fin['Free Cash Flow'].diff(-1).shift(1) / fin['Free Cash Flow']).mean()
  fin['RevGR'] = fin['Total Revenue'].diff(-1).shift(1) / fin['Total Revenue']
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
  if (fin['Cash Dividends Paid']>0).all():
    fin['DivGR'] = fin['Dividend per share'].diff(-1).shift(1) / fin['Dividend per share']
  else:
    fin['DivGR'] = 0
  fin['DivGR'] = fin['DivGR'].fillna(dividend_gr)
  fin['FCFGR'] = fin['Free Cash Flow'].diff(-1).shift(1) / fin['Free Cash Flow']
  fin['FCFGR'] = fin['FCFGR'].fillna(fcf_gr)
  #convert to yearly
  fin['RevGR'] = fin['RevGR'] * 4
  fin['DivGR'] = fin['DivGR'] * 4
  fin['FCFGR'] = fin['FCFGR'] * 4
  fin['rE(Beta)'] = federal_rate + fin['Beta']*(market_growth-federal_rate)-1 #CAPM
  fin['rE(Dividend)'] = fin['Dividend per share']*(1+fin['DivGR'])/fin['Close price'] + fin['DivGR']
  fin['rE'] = np.where((fin['rE(Dividend)']>0.03) & (fin['rE(Dividend)']<0.15), fin['rE(Dividend)'], fin['rE(Beta)'])
  fin['Total debt'] = fin['Current Debt And Capital Lease Obligation'] + fin['Long Term Debt And Capital Lease Obligation']
  fin['rD'] = fin['Interest Expense'] / fin['Total debt']
  #getting rid of negatives
  fin['rD'] = np.where(fin['rD'] > 0, fin['rD'], fin['rD'].shift(-1))
  fin['rD'] = np.where(fin['rD'].isna(), fin['rD'].shift(1), fin['rD'])
  fin['Free Cash Flow'] = np.where(fin['Free Cash Flow'] > 0, fin['Free Cash Flow'], fin['Free Cash Flow'].shift(-1))
  fin['Free Cash Flow'] = np.where(fin['Free Cash Flow'].isnull(), fin['Free Cash Flow'].shift(1), fin['Free Cash Flow'])
  fin['wE'] = fin['Market cap'] / (fin['Market cap'] + fin['Total debt'])
  fin['wD'] = 1 - fin['wE']
  fin['WACC'] = fin['wE'] * fin['rE'] + fin['wD'] * fin['rD'] * (1-fin['Tax Rate For Calcs'])
  fin['Growth'] = np.where( (fin['RevGR'] > terminal_growth) & (fin['FCFGR'] > terminal_growth), np.where(fin['RevGR'] > fin['FCFGR'], fin['FCFGR'], fin['RevGR']), np.where(fin['RevGR'] > terminal_growth, fin['RevGR'], terminal_growth) )
  fin['RollingFCF'] = fin['Free Cash Flow'][::-1].rolling(2).mean()[::-1].fillna(fin['Free Cash Flow'])
  fin['FCF6'] = fin['RollingFCF'] * 4 * (1+fin['Growth'])**6 #*4 to convert to yearly
  fin['Discounted terminal value'] = (fin['FCF6'] / (fin['WACC']-terminal_growth)) / (1+fin['WACC'])**5
  fin['Growing annuity'] = fin['RollingFCF']*(1+fin['Growth'])/(fin['WACC']-fin['Growth']) * (1-((1+fin['Growth'])/(1+fin['WACC']))**5)
  fin['Total value now'] = fin['Discounted terminal value'] + fin['Growing annuity']
  fin['DCF fair share value'] = fin['Total value now'] / fin['Ordinary Shares Number']
  fin['Share px relative to fair value'] = fin['Close price'] / fin['DCF fair share value']

  fin['ticker'] = ticker
  fin = fin.reset_index()
  fin.rename(columns={'index': 'Date'}, inplace=True)

  fin_all = pd.concat([fin_all, fin], axis=0)

<ipython-input-128-bc791d1e9d0a>:67: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
<ipython-input-128-bc791d1e9d0a>:72: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['DivGR'] = fin['DivGR'].fillna(dividend_gr)
<ipython-input-128-bc791d1e9d0a>:74: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)

In [129]:
fin = fin.set_index('Date')

In [130]:
fin[['Free Cash Flow', 'rE', 'rD', 'WACC', 'Interest Expense', 'wE', 'wD', 'RevGR', 'FCFGR', 'Growth', 'Close price', 'DCF fair share value', 'Share px relative to fair value']]

,Free Cash Flow,rE,rD,WACC,Interest Expense,wE,wD,RevGR,FCFGR,Growth,Close price,DCF fair share value,Share px relative to fair value
Date,,,,,,,,,,,,,
2024-12-31,4061000000.0,0.056918,0.009687,0.041604,496000000.0,0.689146,0.310854,0.074640,0.782458,0.074640,55.942513,328.186915,0.170459
2024-09-30,5267000000.0,0.07446,0.009825,0.051993,505000000.0,0.666351,0.333649,0.151362,-0.915891,0.151362,50.60862,290.049312,0.174483
2024-06-30,2064000000.0,0.07446,0.009647,0.047656,521000000.0,0.598967,0.401033,-0.101303,6.207364,0.005000,39.791859,92.16485,0.431747
2024-03-31,2550000000.0,0.07446,0.007403,0.049797,425000000.0,0.640537,0.359463,0.113274,-0.762353,0.113274,50.467918,222.393331,0.226931
2023-12-31,3922000000.0,0.07446,0.007621,0.054058,316000000.0,0.701897,0.298103,0.135227,-1.399286,0.135227,48.307255,272.481977,0.177286


In [112]:
(fin['Free Cash Flow'].diff(-1).shift(1) / fin['Free Cash Flow']).mean()

1.4664174538702037

In [103]:
fin['Free cash flow']

,Free cash flow
Date,
2024-12-31,31346000000.0
2024-09-30,31346000000.0
2024-06-30,37265000000.0
2024-03-31,5305000000.0
2023-12-31,None


In [104]:
np.where(fin['Free cash flow'].isnull(), fin['Free cash flow'].shift(1), fin['Free cash flow'])

array([31346000000.0, 31346000000.0, 37265000000.0, 5305000000.0,
       5305000000.0], dtype=object)